# Langchain Expression Language

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.schema.output_parser import StrOutputParser

In [4]:
model = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

## Composition of the chain using the '|' operator

In [3]:
chain = prompt | model

In [4]:
# Invoke method

chain.invoke({'topic' : 'monkey'})

AIMessage(content='Why don\'t monkeys ever use cell phones?\n\nBecause they already have too many "hang-ups"!', additional_kwargs={}, example=False)

In [5]:
# Stream method

for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why don't bears wear shoes?

Because they have bear feet!

In [6]:
# Batch Method

chain.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!", additional_kwargs={}, example=False),
 AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!", additional_kwargs={}, example=False)]

In [8]:
# Async Stream

async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why don't bears use cell phones?

Because they always get terrible reception in the woods!

In [9]:
# Async Invoke

await chain.ainvoke({"topic": "bears"})

AIMessage(content="Why don't bears like fast food?\n\nBecause they can't catch it!", additional_kwargs={}, example=False)

In [10]:
# Async Batch

await chain.abatch([{"topic": "bears"}])

[AIMessage(content="Why don't bears like fast food? \nBecause they can't catch it!", additional_kwargs={}, example=False)]

## OpenAI Function Calling

In [11]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [15]:
chain.invoke({"topic": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}'}}, example=False)

In [17]:
chain = prompt | model | StrOutputParser()

In [18]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [19]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt 
    | model.bind(function_call= {"name": "joke"}, functions= functions) 
    | JsonOutputFunctionsParser()
)

In [20]:
chain.invoke({"topic": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [21]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
chain = (
    prompt 
    | model.bind(function_call= {"name": "joke"}, functions= functions) 
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [22]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?"

## Multiple Chains

In [25]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

chain1 = prompt1 | model | StrOutputParser()

chain2 = {"city": chain1, "language": itemgetter("language")} | prompt2 | model | StrOutputParser()

chain2.invoke({"person": "obama", "language": "english"})

'Barack Obama, the 44th President of the United States, was born in Honolulu, Hawaii, which is part of the United States.'

In [6]:
import langchain
langchain.debug = True

In [7]:
from langchain.schema.runnable import RunnableMap

prompt1 = ChatPromptTemplate.from_template("generate a random color")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}")
prompt3 = ChatPromptTemplate.from_template("what is countries flag that has the color: {color}")
prompt4 = ChatPromptTemplate.from_template("What is the color of {fruit} and {country}")
chain1 = prompt1 | model | StrOutputParser()
chain2 = RunnableMap(steps={"color": chain1}) | {
    "fruit": prompt2 | model | StrOutputParser(),
    "country": prompt3 | model | StrOutputParser(),
} | prompt4 



chain2.invoke({})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {}
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "generate a rand

ChatPromptValue(messages=[HumanMessage(content="What is the color of The fruit that corresponds to the color #FF00FF is commonly known as a dragon fruit. Dragon fruits have a vibrant pink or magenta exterior with green scales. and The country's flag that has the color #FF00FF (bright pink or magenta) is the flag of Suriname.", additional_kwargs={}, example=False)])